In [1]:
from kaggle_environments import make
import json

Loading environment football failed: No module named 'gfootball'


# Create the environment. 

You can also specify configurations for seed and loglevel as shown below. If not specified, a random seed is chosen. 

- loglevel default is 0
- 1 is for errors
- 2 is for match warnings such as units colliding, invalid commands (recommended)
- 3 for info level
- 4 for everything (not recommended)
 
set annotations True so annotation commands are drawn on visualizer

set debug to True so print statements get shown

In [2]:
env = make(
    "lux_ai_2021", 
    configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, 
    debug=True,
)

# Create agent

In [3]:
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

# we declare this global game_state object so that state 
# persists across turns so we do not need to reinitialize it all the time
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)
        actions.append(annotate.circle(0, 0))
    return actions

# Run a match between two agents

In [4]:
steps = env.run([agent, "simple_agent"])

Agent is running!


# Render the game

In [ ]:
#  env.render(mode="ipython", width=1200, height=800)

# Create support function

In [5]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

# View some data about game statement

## Game and agents

In [6]:
print('Game turn is: {}'.format(game_state.turn))

Game turn is: 359


In [7]:
print('Agents ids. Player: {}. Opponent: {}'.format(game_state.id, (game_state.id + 1) % 2))

Agents ids. Player: 0. Opponent: 1


In [8]:
print('players objects:')
print('Player:', game_state.players[game_state.id])
print('Opponent:', game_state.players[(game_state.id + 1) % 2])

players objects:
Player: <lux.game_objects.Player object at 0x7f1cceebd7c0>
Opponent: <lux.game_objects.Player object at 0x7f1ccedd4160>


## Map

In [9]:
print('map size properties: {} x {}'.format(game_state.map.width, game_state.map.height))
print('Map object : ', game_state.map)
print('Map cell by coordinates: ', game_state.map.get_cell(12, 24))

map size properties: 32 x 32
Map object :  <lux.game_map.GameMap object at 0x7f1ccedcd700>
Map cell by coordinates:  <lux.game_map.Cell object at 0x7f1ccedc9b80>


In [10]:
print('Position object on map in sell 1,1: ,', Position(1, 1))
print('Map cell by position: ', game_state.map.get_cell_by_pos(Position(1, 1)))

Position object on map in sell 1,1: , (1, 1)
Map cell by position:  <lux.game_map.Cell object at 0x7f1ca9136940>


## Position

In [11]:
print('"is_adjacent(pos: Position) -> bool" returns true if this Position is adjacent to pos. False otherwise\n')
print('ajaced position', Position(1, 2).is_adjacent(Position(1, 1)))
print('same position', Position(1, 1).is_adjacent(Position(1, 1)))
print('not ajaced position', Position(12, 12).is_adjacent(Position(1, 1)))

"is_adjacent(pos: Position) -> bool" returns true if this Position is adjacent to pos. False otherwise

ajaced position True
same position True
not ajaced position False


In [12]:
print('"equals(pos: Position) -> bool" - returns true if this Position is equal to the other pos object by \
      checking x, y coordinates. False otherwise\n')
print('same position is equals', Position(1, 1).equals(Position(1, 1)))

"equals(pos: Position) -> bool" - returns true if this Position is equal to the other pos object by       checking x, y coordinates. False otherwise

same position is equals True


In [13]:
print('"translate(direction: DIRECTIONS, units: int) -> Position" - returns the Position equal to going in \
a direction units number of times from this Position\n')

pos = Position(1, 1).translate(Constants.DIRECTIONS.EAST, 5)
print('5 times EAST from 1,1 position is:', pos)
print('then 1 times NORTH position is:', pos.translate(Constants.DIRECTIONS.NORTH, 1))

"translate(direction: DIRECTIONS, units: int) -> Position" - returns the Position equal to going in a direction units number of times from this Position

5 times EAST from 1,1 position is: (6, 1)
then 1 times NORTH position is: (6, 0)


![game_board](../game_board.png)

In [14]:
print('"distance_to(pos: Position) -> float" - returns the Manhattan (rectilinear) distance \
from this Position to pos\n')

print('Manhattam distance between 1,1 and 7,11 is: ', Position(1, 1).distance_to(Position(7, 11)))

"distance_to(pos: Position) -> float" - returns the Manhattan (rectilinear) distance from this Position to pos

Manhattam distance between 1,1 and 7,11 is:  16


In [15]:
print('"direction_to(target_pos: Position) -> DIRECTIONS" - returns the direction that would move you \
closest to target_pos from this Position if you took a single step. In particular, will \
return DIRECTIONS.CENTER if this Position is equal to the target_pos. Note that this does not \
check for potential collisions with other units but serves as a basic pathfinding method\n')

print('Direction between 1,1 to 7,11 is: ', Position(1, 1).direction_to(Position(7, 11)))
assert Position(1, 1).direction_to(Position(7, 11)) == Constants.DIRECTIONS.EAST

"direction_to(target_pos: Position) -> DIRECTIONS" - returns the direction that would move you closest to target_pos from this Position if you took a single step. In particular, will return DIRECTIONS.CENTER if this Position is equal to the target_pos. Note that this does not check for potential collisions with other units but serves as a basic pathfinding method

Direction between 1,1 to 7,11 is:  e


## Cell

- pos: Position
- resource: Resource - contains details of a Resource at this Cell. This may be equal to None or null equivalents in other languages. You should always use the function has_resource to check if this Cell has a Resource or not
- road: float - the amount of Cooldown subtracted from a Unit's Cooldown whenever they perform an action on this tile. If there are roads, the more developed the road, the higher this Cooldown rate value is. Note that a Unit will always gain a base Cooldown amount whenever any action is performed.
- citytile: CityTile - the citytile that is on this Cell. Equal to none or null equivalents in other languages if there is no CityTile here.

has_resource() -> bool - returns true if this Cell has a non-depleted Resource, false otherwise

In [16]:
print('Cell object: ', Cell(1, 1))
print('Cell position: ', Cell(1, 1).pos)
print('Cell resourse details: ', Cell(1, 1).resource)
print('Cell road details: ', Cell(1, 1).road)
print('Cell citytitle details: ', Cell(1, 1).citytile)
print('Cell has resource: ', Cell(1, 1).has_resource())

Cell object:  <lux.game_map.Cell object at 0x7f1ccee61ac0>
Cell position:  (1, 1)
Cell resourse details:  None
Cell road details:  0
Cell citytitle details:  None
Cell has resource:  False


## City

- cityid: str - the id of this City. Each City id in the game is unique and will never be reused by new cities
- team: int - the id of the team this City belongs to.
- fuel: float - the fuel stored in this City. This fuel is consumed by all CityTiles in this City during each turn of night.
- citytiles: list[CityTile] - a list of CityTile objects that form this one City collectively. A City is defined as all CityTiles that are connected via adjacent CityTiles.

get_light_upkeep() -> float - returns the light upkeep per turn of the City. Fuel in the City is subtracted by the light upkeep each turn of night.

In [17]:
cityes = game_state.players[game_state.id].cities
print(cityes)
print('city object: ', cityes['c_1'])
print('city id: ', cityes['c_1'].cityid)
print('city team id: ', cityes['c_1'].team)
print('city fuel stored: ', cityes['c_1'].fuel)
print('city tiles list: ', cityes['c_1'].citytiles)
print('city light upkeep coast: ', cityes['c_1'].get_light_upkeep())

{'c_1': <lux.game_objects.City object at 0x7f1ca920a6a0>}
city object:  <lux.game_objects.City object at 0x7f1ca920a6a0>
city id:  c_1
city team id:  0
city fuel stored:  137.0
city tiles list:  [<lux.game_objects.CityTile object at 0x7f1ca920a340>]
city light upkeep coast:  23.0


## CityTile

- cityid: str - the id of the City this CityTile is a part of. Each City id in the game is unique and will never be reused by new cities
- team: int - the id of the team this CityTile belongs to.
- pos: Position - the Position of this City on the map
- cooldown: float - the current Cooldown of this City.

Methods:

can_act() -> bool - whether this City can perform an action this turn, which is when the Cooldown is less than 1

research() -> str - returns the research action

build_worker() -> str - returns the build worker action. When applied and requirements are met, a worker will be built at the City.

build_cart() -> str - returns the build cart action. When applied and requirements are met, a cart will be built at the City.

In [18]:
# snippet to find the closest city tile to a position
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

In [19]:
closest_city_tile = find_closest_city_tile(Cell(1, 1).pos, game_state.players[game_state.id])

In [20]:
print('city_tile id: ', closest_city_tile.cityid)
print('city_tile team id: ', closest_city_tile.team)
print('city_tile position: ', closest_city_tile.pos)
print('city_tile current cooldown: ', closest_city_tile.cooldown)

city_tile id:  c_1
city_tile team id:  0
city_tile position:  (3, 27)
city_tile current cooldown:  0.0


In [21]:
print('city_tile can perfor, action this turn: ', closest_city_tile.can_act())
print('city_tile make research action: ', closest_city_tile.research())
print('city_tile build worker: ', closest_city_tile.build_worker())
print('city_tile build cart: ', closest_city_tile.build_cart())
print('cant build because we have one city with one tile and one worker - its limited')

city_tile can perfor, action this turn:  True
city_tile make research action:  r 3 27
city_tile build worker:  bw 3 27
city_tile build cart:  bc 3 27
cant build because we have one city with one tile and one worker - its limited


## Unit

Properties:

- pos: Position - the Position of this Unit on the map
- team: int - the id of the team this Unit belongs to.
- id: str - the id of this Unit. This is unique and cannot be repeated by any other Unit or City
- cooldown: float - the current Cooldown of this Unit. Note that when this is less than 1, the Unit can perform an action
- cargo.wood: int - the amount of wood held by this Unit
- cargo.coal: int - the amount of coal held by this Unit
- cargo.uranium: int - the amount of uranium held by this Unit

Methods:

- get_cargo_space_left(): int - returns the amount of space left in the cargo of this Unit. Note that any Resource takes up the same space, e.g. 70 wood takes up as much space as 70 uranium, but 70 uranium would produce much more fuel than wood when deposited at a City
- can_build(game_map: GameMap): bool - returns true if the Unit can build a City on the tile it is on now. False otherwise. Checks that the tile does not have a Resource over it still and the Unit has a Cooldown of less than 1
- can_act(): bool - returns true if the Unit can perform an action. False otherwise. Essentially checks whether the Cooldown of the Unit is less than 1
- move(dir): str - returns the move action. When applied, Unit will move in the specified direction by one Unit, provided there are no other units in the way or opposition cities. (Units can stack on top of each other however when over a friendly City)
- transfer(dest_id, resourceType, amount): str - returns the transfer action. Will transfer from this Unit the selected Resource type by the desired amount to the Unit with id dest_id given that both units are adjacent at the start of the turn. (This means that a destination Unit can receive a transfer of resources by another Unit but also move away from that Unit)
- build_city(): str - returns the build City action. When applied, Unit will try to build a City right under itself provided it is an empty tile with no City or resources and the worker is carrying 100 units of resources. All resources are consumed if the city is succesfully built.
- pillage(): str - returns the pillage action. When applied, Unit will pillage the tile it is currently on top of and remove 0.5 of the road level.

In [24]:
player = game_state.players[game_state.id]
unit = player.units[0]
print('First unit object of player: ', unit)

First unit object of player:  <lux.game_objects.Unit object at 0x7f1ca9194ca0>


In [29]:
print(
    'Properties:\n\
    position: {0}\n\
    team: {1}\n\
    id: {2}\n\
    cooldown: {3}\n\
    cargo of wood/coal/uranium: {4}/{5}/{6}'.format(
        unit.pos,
        unit.team,
        unit.id,
        unit.cooldown,
        unit.cargo.wood,
        unit.cargo.coal,
        unit.cargo.uranium,
        )
    )

Properties:
    position: (3, 27)
    team: 0
    id: u_1
    cooldown: 0.0
    cargo of wood/coal/uranium: 0/0/0


In [36]:
print(
    'Methods:\n\
    cargo space: {0}\n\
    can build on that title: {1}\n\
    can act: {2}\n\
    transfer to chosen direction action: {3}\n\
    build city action: {4}\n\
    pillage action: {5}\n\
    The actions performeds if it possible'.format(
        unit.get_cargo_space_left(),
        unit.can_build(game_state.map),
        unit.can_act(),
        unit.move(Constants.DIRECTIONS.NORTH),
        unit.build_city(),
        unit.pillage(),
        )
    )

Methods:
    cargo space: 100
    can build on that title: False
    can act: True
    transfer to chosen direction action: m u_1 n
    build city action: bcity u_1
    pillage action: p u_1
    The actions performeds if it possible


## Player

This contains information on a particular player of a particular team.

Properties:

- team: int - the team id of this player
- research_points: int - the current total number of research points the player's team has
- units: list[Unit] - a list of every Unit owned by this player's team.

- cities: Dict[str, City] - a dictionary / map mapping City id to each separate City owned by this player's team. To get the individual CityTiles, you will need to access the citytiles property of the City.

Methods:

- researched_coal() - bool - whether or not this player's team has researched coal and can mine coal.
- researched_uranium() - bool - whether or not this player's team has researched uranium and can mine uranium.

In [40]:
print(
    'Player properties: \n\
    team: {}\n\
    research_points: {}\n\
    list of unit objects: {}\n\
    dict of cities objects with id-s as keys: {}'.format(
        player.team,
        player.research_points,
        player.units,
        player.cities,
    )
)

Player properties: 
    team: 0
    research_points: 0
    list of unit objects: [<lux.game_objects.Unit object at 0x7f1ca9194ca0>]
    dict of cities objects with id-s as keys: {'c_1': <lux.game_objects.City object at 0x7f1ca920a6a0>}


In [41]:
print(
    'Player methods:\n\
    is coal researched: {}\n\
    is uranium researched: {}'.format(
        player.researched_coal(),
        player.researched_uranium()
        )
    )

Player methods:
    is coal researched: False
    is uranium researched: False


## Methods

- circle(x: int, y: int) -> str - returns the draw circle annotation action. Will draw a unit sized circle on the visualizer at the current turn centered at the Cell at the given x, y coordinates
- x(x: int, y: int) -> str - returns the draw X annotation action. Will draw a unit sized X on the visualizer at the current turn centered at the Cell at the given x, y coordinates
- line(x1: int, y1: int, x2: int, y2: int) -> str - returns the draw line annotation action. Will draw a line from the center of the Cell at (x1, y1) to the center of the Cell at (x2, y2)
- text(x: int, y: int, message: str, fontsize: int = 16) -> str: - returns the draw text annotation action. Will write text on top of the tile at (x, y) with the particular message and fontsize

sidetext(message: str) -> str: - returns the draw side text annotation action. Will write text that is displayed on that turn on the side of the visualizer

Note that all of these will be colored according to the team that created the annotation (blue or orange)

In [42]:
# lets look at some of the resources found
resource_tiles = find_resources(game_state)
cell = resource_tiles[0]
print("Cell at", cell.pos, "has")
print(cell.resource.type, cell.resource.amount)

Cell at (8, 0) has
coal 369


In [43]:
# lets see if we do find some close resources
cell = find_closest_resources(Position(1, 1), game_state.players[0], resource_tiles)
print("Closest resource at", cell.pos, "has")
print(cell.resource.type, cell.resource.amount)

Closest resource at (4, 10) has
wood 500


# Full data for Submission

In [ ]:
%%writefile agent.py
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

### Define helper functions

# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    resource_tiles = find_resources(game_state)
    
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            # we want to mine only if there is space left in the worker's cargo
            if unit.get_cargo_space_left() > 0:
                # find the closest resource if it exists to this unit
                closest_resource_tile = find_closest_resources(unit.pos, player, resource_tiles)
                if closest_resource_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_resource_tile.pos))
                    actions.append(action)
            else:
                # find the closest citytile and move the unit towards it to drop resources to a citytile to fuel the city
                closest_city_tile = find_closest_city_tile(unit.pos, player)
                if closest_city_tile is not None:
                    # create a move action to move this unit in the direction of the closest resource tile and add to our actions list
                    action = unit.move(unit.pos.direction_to(closest_city_tile.pos))
                    actions.append(action)
    
    return actions

In [ ]:
replay = env.toJSON()
with open("replays/replay.json", "w") as f:
    json.dump(replay, f)

## Create a submission

In [ ]:
!sh make_sub.sh

## Submit

In [ ]:
!kaggle competitions submit -c lux-ai-2021 -f submission.py -m "submission"

## Suggestions / Strategies

There are a lot of places that could be improved with the agent we have in this tutorial notebook. Here are some!

- Using the build city action to build new cities and thus build new units
- Having cities perform research each turn to unlock new resources
- Writing collision-free code that lets units move smoothly around and through each other when navigating to targets
- Mining resources near your opponent's citytiles so they have less easy access to resources
- Using carts to deliver resources from far away clusters of wood, coal, uranium to a city in need
- Sending worker units over to the opponent's roads and pillaging them to slow down their agent
- Optimizing over how much to mine out of forests before letting them regrow so you can build more cities and get sustainable fuel

In [3]:
import pandas as pd